# Deep Convolutional GAN 

Copyright: Tensorflow gan examples

reference [link](https://github.com/tensorflow/tensorflow/blob/r1.13/tensorflow/contrib/eager/python/examples/generative_examples/dcgan.ipynb)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import Progbar

In [3]:
tf.enable_eager_execution()

In [5]:
from dcgan_models import make_generator_model, make_discriminator_model

#### Models 

In [6]:
Generator = make_generator_model(noise_dim=100)
Generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12544)             1254400   
_________________________________________________________________
batch_normalization (BatchNo (None, 12544)             50176     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 12544)             0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 128)         819200    
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 128)         512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 128)         0 

In [7]:
Discriminator = make_discriminator_model(image_shape=[28,28,1], noise_dim=100)
Discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)            

#### Loss Functions

In [ ]:
def generator_loss(generated_output):
    return tf.losses.sigmoid_cross_entropy(tf.ones_like(generated_output), generated_output)

def discriminator_loss(real_output, generated_output):
    real_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.ones_like(real_output), logits=real_output)
    generated_loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.zeros_like(generated_output), logits=generated_output)
    total_loss = real_loss + generated_loss
    return total_loss

#### Optimizers 

In [ ]:
generator_optimizer     = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5)
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5)

#### Checkpoints 

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 Generator=Generator,
                                 Discriminator=Discriminator)

In [ ]:
num_examples_to_generate = 9

# We'll re-use this random vector used to seed the Generator so
# it will be easier to see the improvement over time.
random_vector_for_generation = tf.random_normal([num_examples_to_generate, NOISE_DIM])

### Dataset

In [ ]:
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [ ]:
BUFFER_SIZE = len(train_labels)
BATCH_SIZE  = 50
NUM_BATCHES = BUFFER_SIZE//BATCH_SIZE
EPOCHS = 50

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).repeat(EPOCHS).batch(BATCH_SIZE)

### Training  


In [ ]:
# one iteration of training G and D on a minibatch
# TODO: k iterations of D training for 1 iteration of G and/or threshold for loss
@tf.function
def train_step(images):
    # generating noise from a normal distribution
    noise = tf.random_normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = Generator(noise, training=True)

        real_output = Discriminator(images, training=True)
        generated_output = Discriminator(generated_images, training=True)

        gen_loss = generator_loss(generated_output)
        disc_loss = discriminator_loss(real_output, generated_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, Generator.variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, Discriminator.variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, Generator.variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, Discriminator.variables))
    return gen_loss, disc_loss

In [ ]:
# compile the function into tf graph
train_step = tf.contrib.eager.defun(train_step)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # make sure the training parameter is set to False because we
    # don't want to train the batchnorm layer when doing inference.
    predictions = model(test_input, training=False)
    
    fig = plt.figure(figsize=(9,9))
      
    for i in range(predictions.shape[0]):
        plt.subplot(3, 3, i+1)
        plt.imshow(predictions[i, :, :, 0], cmap='gray')
        plt.axis('off')
          
    plt.savefig('big_training_images/image_at_epoch_{:04d}.png'.format(epoch))

In [ ]:
def train(dataset, epochs, initial_epoch=0):  
    # TODO: log loss values to csv/text file
    # TODO: load from saved checkpoints 
    for epoch in range(initial_epoch, epochs+initial_epoch):
        # TODO: try main loop over infinite dataset iterator
        progress_bar = Progbar(target=STEPS_PER_EPOCH)
        print('Epoch {} of {}'.format(epoch + 1, epochs+initial_epoch))
        D_loss = []
        G_loss = []
        batch_count = 0
        for images in dataset:
            if batch_count==STEPS_PER_EPOCH:
                break
            g_loss, d_loss = train_step(images)
            D_loss.append(d_loss)
            G_loss.append(g_loss)
            batch_count +=1
            progress_bar.update(batch_count)


        generate_and_save_images(Generator, epoch + 1, random_vector_for_generation)

        # saving (checkpoint) the model every 10 epochs
        if (epoch + 1) % 10 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
            Generator.save('generator.h5')
            Discriminator.save('discriminator.h5')

        print('Discriminator loss: {},  Generator loss: {}'.format(np.mean(D_loss), np.mean(G_loss)))
    # generating after the final epoch
    generate_and_save_images(Generator, epochs+initial_epoch, random_vector_for_generation)

In [ ]:
train(train_dataset, EPOCHS, initial_epoch=0)

In [ ]:
Generator.save('generator.h5')
Discriminator.save('discriminator.h5')